In [30]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')
import datetime as dt
from sublimpy import utils
import pytz

# Open SOS Measurement Dataset

In [31]:
ls -lah | grep parquet

-rw-r--r--@    1 elischwat  staff   202M Apr  3 18:07 coare_model_results.parquet
-rw-r--r--@    1 elischwat  staff   116M Mar  6  2024 model_results.parquet
-rw-r--r--@    1 elischwat  staff   138M Sep 12 14:53 tidy_df_10min.parquet
-rw-r--r--     1 elischwat  staff    57M Jul 30 15:16 tidy_df_20221101_20230619_noplanar_fit.parquet
-rw-r--r--     1 elischwat  staff    57M Jul 18 15:41 tidy_df_20221101_20230619_planar_fit.parquet
-rw-r--r--@    1 elischwat  staff    57M Aug 21 14:30 tidy_df_20221101_20230619_planar_fit_multiplane.parquet
-rw-r--r--@    1 elischwat  staff    58M Aug 29 11:38 tidy_df_20221101_20230619_planar_fit_multiplane_flaggeddataremains.parquet
-rw-r--r--@    1 elischwat  staff    38M Jul  2 22:09 tidy_df_20221130_20230509_noplanar_fit.parquet
-rw-r--r--     1 elischwat  staff   257M Feb 20  2024 tidy_df_20221130_20230509_noplanar_fit_ARCHIVE.parquet
-rw-r--r--@    1 elischwat  staff    50M Apr  3 18:06 tidy_df_20221130_20230509_noplanar_fit_clean.parquet
-rw-r--r--

In [32]:
start_date = '20221101'
end_date = '20230619'

data_start_date = '20221130'
data_end_date = '20230508'
# open files
# tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet')

In [33]:
tidy_df = utils.modify_df_timezone(
    tidy_df,
    pytz.UTC,
    'US/Mountain',   
)
tidy_df = tidy_df.set_index('time').sort_index().loc[data_start_date:data_end_date].reset_index()

In [34]:
# quick way to get variable info if we want it 
# import xarray as xr
# ds = xr.open_dataset("/storage/elilouis/sublimationofsnow/sosnoqc/isfs_20221228.nc")
# ds['SWE_p2_c']

In [35]:
src = tidy_df[tidy_df.variable.isin([ 'Tsurf_c', 'Tsurfplanck_c' ])].pivot_table(index='time', values='value', columns='variable')
alt.Chart(
    src
).mark_circle(size=1).encode(
    alt.X('Tsurf_c:Q'),
    alt.Y('Tsurfplanck_c:Q'),
).properties(
    title=f"Mean difference: {np.mean(src.Tsurf_c - src.Tsurfplanck_c)}"
)

alt.Chart(...)

# Make plot showing cumulative sublimation

In [36]:
seconds_in_timestep = 60*30
from metpy.constants import density_water

In [37]:
rnet_chart_with_axis = alt.Chart(
    tidy_df.query("variable == 'Rnet_9m_d'")
).transform_window(
    frame = [-12, 12],
    rolling_mean = 'mean(value)'
).mark_line(opacity=0.5).encode(
    alt.X("time:T"),
    alt.Y("rolling_mean:Q").title(["Net radiation", "(W/m^2)"])
)
rnet_chart_with_axis

alt.Chart(...)

In [38]:
bs_flux_chart = alt.Chart(
    tidy_df[tidy_df.measurement == 'snow flux'].query("value > 0")
).mark_circle(opacity=0.5).encode(
    # alt.X("time:T").axis(labels=False).title(None),
    alt.X("time:T"),
    alt.Y("value:Q").title(["Blowing snow flux", "(g/m²/s)"]).scale(type='log'),
    alt.Color('height:N')
)
bs_flux_chart

alt.Chart(...)

In [39]:
tidy_df.query("variable == 'RH_3m_c'")

,time,variable,value,height,tower,measurement
682,2022-11-30 00:00:00,RH_3m_c,76.349220,3.0,c,RH
1111,2022-11-30 00:30:00,RH_3m_c,73.764297,3.0,c,RH
2561,2022-11-30 01:00:00,RH_3m_c,74.464081,3.0,c,RH
2813,2022-11-30 01:30:00,RH_3m_c,75.650993,3.0,c,RH
4165,2022-11-30 02:00:00,RH_3m_c,75.693504,3.0,c,RH
...,...,...,...,...,...,...
6630162,2023-05-08 21:30:00,RH_3m_c,37.994419,3.0,c,RH
6630414,2023-05-08 22:00:00,RH_3m_c,38.012474,3.0,c,RH
6631859,2023-05-08 22:30:00,RH_3m_c,47.978481,3.0,c,RH
6632662,2023-05-08 23:00:00,RH_3m_c,49.237717,3.0,c,RH


In [40]:
rnet_chart = alt.Chart(
    tidy_df.query("variable == 'RH_3m_c'")
).transform_window(
    frame = [-12, 12],
    rolling_mean = 'mean(value)'
).mark_line(opacity=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
)
rnet_chart


alt.Chart(...)

In [41]:
rnet_chart = alt.Chart(
    tidy_df.query("variable == 'Rnet_9m_d'")
).transform_window(
    frame = [-12, 12],
    rolling_mean = 'mean(value)'
).mark_line(opacity=0.5).encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("rolling_mean:Q").title(["Net radiation", "(W/m²)"])
)
rnet_chart


alt.Chart(...)

In [42]:
tidy_df.query("measurement == 'SWE'").groupby('variable')['value'].max()

variable
SWE_p1_c    682.850098
SWE_p2_c    436.592560
SWE_p3_c    444.716095
SWE_p4_c    549.908386
Name: value, dtype: float64

In [43]:
snowpillow_chart = alt.Chart(
    tidy_df.query("variable == 'SWE_p2_c'").dropna()
).transform_window(
    frame = [-48, 48],
    rolling_median = 'median(value)'
).mark_line(color='black').encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("rolling_median:Q").title(["Snow water", "equivalent (mm)"])
).properties(height = 83)
snowpillow_chart

alt.Chart(...)

In [44]:
tidy_df[tidy_df.variable.isin([
        'SWE_p1_c', 'SWE_p2_c', 'SWE_p3_c', 'SWE_p4_c'
    ])][
        tidy_df.time < '20230415'
    ].groupby('variable').value.max()

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_97466/1114611917.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tidy_df[tidy_df.variable.isin([


variable
SWE_p1_c    650.430420
SWE_p2_c    436.592560
SWE_p3_c    444.716095
SWE_p4_c    538.970276
Name: value, dtype: float64

In [45]:
alt.Chart(
    tidy_df[tidy_df.variable.isin([
        'SWE_p1_c', 'SWE_p2_c', 'SWE_p3_c', 'SWE_p4_c'
    ])][
        tidy_df.time < '20230415'
    ]
).transform_window(
    frame = [-48, 48],
    rolling_median = 'median(value)',
    groupby=['variable']
).mark_line(color='black').encode(
    alt.X("time:T"),
    alt.Y("rolling_median:Q").title(["Snow water", "equivalent (mm)"]),
    alt.Color('variable:N')
)


/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_97466/3976729874.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tidy_df[tidy_df.variable.isin([


alt.Chart(...)

In [46]:
tidy_df[tidy_df.variable.isin([
    'SWE_p1_c', 'SWE_p2_c', 'SWE_p3_c', 'SWE_p4_c'
])].query("time < 20240410")

,time,variable,value,height,tower,measurement
74,2022-11-30 00:00:00,SWE_p4_c,34.155613,0.0,d,SWE
116,2022-11-30 00:00:00,SWE_p3_c,38.953293,0.0,ue,SWE
123,2022-11-30 00:00:00,SWE_p2_c,24.134979,0.0,c,SWE
187,2022-11-30 00:00:00,SWE_p1_c,28.701609,0.0,uw,SWE
982,2022-11-30 00:30:00,SWE_p1_c,28.916878,0.0,uw,SWE
...,...,...,...,...,...,...
6632812,2023-05-08 23:00:00,SWE_p4_c,375.274261,0.0,d,SWE
6633168,2023-05-08 23:30:00,SWE_p1_c,330.533508,0.0,uw,SWE
6633396,2023-05-08 23:30:00,SWE_p3_c,189.031097,0.0,ue,SWE
6633450,2023-05-08 23:30:00,SWE_p4_c,374.843018,0.0,d,SWE


In [47]:
measured_results_mm = tidy_df[tidy_df.variable.isin([
    'w_h2o__2m_c',
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',

    'w_h2o__3m_ue',
    'w_h2o__10m_ue',

    'w_h2o__3m_uw',
    'w_h2o__10m_uw',

    'w_h2o__3m_d',
    'w_h2o__10m_d',
])][
    ['time', 'value', 'variable']
]
measured_results_mm.columns = ['time', 'measured', 'variable']
measured_results_mm = measured_results_mm.pivot(
    index = 'time',
    columns = 'variable',
    values = 'measured'
)
# convert to mm
measured_results_mm = measured_results_mm*seconds_in_timestep/density_water.magnitude
measured_results_cumsum = measured_results_mm.cumsum().reset_index()
measured_results_cumsum = measured_results_cumsum.melt(id_vars='time')
measured_results_cumsum['height'] = measured_results_cumsum['variable'].apply(lambda s: int(s.split('__')[1].split('m')[0]))
measured_results_cumsum['tower'] = measured_results_cumsum['variable'].apply(lambda s: s.split('__')[1].split('_')[-1])
measured_results_cumsum['tower-height'] = measured_results_cumsum['tower'] + '-' + measured_results_cumsum['height'].astype('str')
measured_results_cumsum


measured_results_cumsum

,time,variable,value,height,tower,tower-height
0,2022-11-30 00:00:00,w_h2o__10m_c,0.002158,10,c,c-10
1,2022-11-30 00:30:00,w_h2o__10m_c,0.003131,10,c,c-10
2,2022-11-30 01:00:00,w_h2o__10m_c,0.002010,10,c,c-10
3,2022-11-30 01:30:00,w_h2o__10m_c,0.002767,10,c,c-10
4,2022-11-30 02:00:00,w_h2o__10m_c,0.002208,10,c,c-10
...,...,...,...,...,...,...
92131,2023-05-08 21:30:00,w_h2o__5m_c,38.349439,5,c,c-5
92132,2023-05-08 22:00:00,w_h2o__5m_c,38.347429,5,c,c-5
92133,2023-05-08 22:30:00,w_h2o__5m_c,38.354877,5,c,c-5
92134,2023-05-08 23:00:00,w_h2o__5m_c,38.355376,5,c,c-5


In [48]:
src = measured_results_cumsum.dropna()
src = src[src.time <= '20230509']
alt.Chart(src).transform_filter(
    alt.datum.height > 1
).mark_line(strokeWidth=1.5).transform_window(
    rolling_median = "median(value)",
    groupby = ["height", "tower-height"],
    frame=[-2,2]
).encode(
    alt.X("time:T"),
    alt.Y("rolling_median:Q").title("Cumulative sublimation (mm)"),
    alt.Color("height:O").scale(scheme = 'turbo'),
    detail = "tower-height"
)

alt.Chart(...)

In [49]:
cum_sub_chart = alt.Chart(measured_results_cumsum.dropna()).transform_filter(
    alt.datum.height > 1
).mark_line(strokeWidth=1.5).transform_window(
    rolling_median = "median(value)",
    groupby = ["height", "tower-height"],
    frame=[-2,2]
).encode(
    alt.X("time:T"),
    alt.Y("rolling_median:Q").title("Cumulative sublimation (mm)"),
    alt.Color("height:O").scale(scheme = 'turbo'),
    detail = "tower-height"
)

cum_sub_chart_no_xaxislabels = alt.Chart(measured_results_cumsum.dropna()).transform_filter(
    alt.datum.height > 1
).mark_line(strokeWidth=1.5).transform_window(
    rolling_median = "median(value)",
    groupby = ["height", "tower-height"],
    frame=[-2,2]
).encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("rolling_median:Q").title("Cumulative sublimation (mm)"),
    alt.Color("height:O").scale(scheme = 'turbo'),
    detail = "tower-height"
)

In [50]:
cum_sub_and_snowpack_swe_chart = (snowpillow_chart & cum_sub_chart).resolve_scale(
    x='shared', y='independent', color='independent'
)

cum_sub_and_snowpack_swe_chart.save("cum-sub-and-snowpack-swe.png", ppi=200)
cum_sub_and_snowpack_swe_chart

alt.VConcatChart(...)

# Make plot showing daily sublimation by clear/blowing snow 

## Calculate daily sublimation

In [51]:
import metpy.constants
seconds_per_timestep = 60*30

In [52]:
daily_sub_by_blowingsnow_src = tidy_df[
    tidy_df.variable.isin(['w_h2o__3m_c', 'SF_avg_1m_ue', 'SF_avg_2m_ue'])
].pivot_table(
    values = 'value',
    index = 'time',
    columns = 'variable'
)
daily_sub_by_blowingsnow_src['SF_avg_max_ue'] = daily_sub_by_blowingsnow_src[['SF_avg_1m_ue', 'SF_avg_2m_ue']].max(axis=1)
daily_sub_by_blowingsnow_src['blowing snow'] = daily_sub_by_blowingsnow_src['SF_avg_max_ue'] > 0
daily_sub_by_blowingsnow_src['Sublimation (mm)'] = daily_sub_by_blowingsnow_src[
    'w_h2o__3m_c'
]*seconds_per_timestep / metpy.constants.density_water.magnitude# calculate daily sublimation
daily_sub_by_blowingsnow_src = daily_sub_by_blowingsnow_src.groupby([pd.Grouper(freq='1440Min'), 'blowing snow']).sum()
daily_sub_by_blowingsnow_src = daily_sub_by_blowingsnow_src.reset_index()
daily_sub_by_blowingsnow_src

variable,time,blowing snow,SF_avg_1m_ue,SF_avg_2m_ue,w_h2o__3m_c,SF_avg_max_ue,Sublimation (mm)
0,2022-11-30,False,0.000000,0.000000,-0.020312,0.000000,-0.036563
1,2022-12-01,False,0.000000,0.000000,0.064413,0.000000,0.115945
2,2022-12-02,False,0.000000,0.000000,0.001230,0.000000,0.002215
3,2022-12-02,True,3.015333,3.996667,0.199097,4.093667,0.358383
4,2022-12-03,False,0.000000,0.000000,0.006118,0.000000,0.011012
...,...,...,...,...,...,...,...
261,2023-05-05,True,0.000000,0.001667,0.010767,0.001667,0.019382
262,2023-05-06,False,0.000000,0.000000,0.369746,0.000000,0.665559
263,2023-05-06,True,0.063333,0.121667,0.002010,0.123333,0.003618
264,2023-05-07,False,0.000000,0.000000,0.127209,0.000000,0.228982


## Plot

In [53]:
daily_sub_chart = alt.Chart(daily_sub_by_blowingsnow_src).mark_bar(width=2.5).encode(
    alt.X("time:T", title=None).axis(labels=False),
    alt.Y("Sublimation (mm):Q").title(["Daily sublimation", "(mm)"]), 
    alt.Color("blowing snow:N").scale(domain=[True, False], range=['#000000', '#C0C0C0']),
    tooltip='time:T'
).properties(height = 100, width=500)

In [54]:
(daily_sub_chart).resolve_scale(y='independent').configure_axis(grid=False)

alt.Chart(...)

# Make stability regime hovmoller plot

## Using temp gradient (pot virtal temp gradient)

In [55]:
src = tidy_df.query("variable == 'temp_gradient_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()

# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
# round for values that are equal to or greater than 0 (stable cases)
stable_src = src[src.value >= -0.01]
stable_src['value'] = stable_src['value'].round(1)

unstable_src = src[src.value < -0.01]
unstable_src['value'] = -0.1

src = pd.concat([stable_src, unstable_src])

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_97466/836200928.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stable_src['value'] = stable_src['value'].round(1)
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_97466/836200928.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unstable_src['value'] = -0.1


In [56]:
domain = np.linspace(-0.1,0.8, 10).round(1)
range = [
    '#ff2600', #red
    '#ffffff', #blue scale, increasinlgy blue
    '#e6ebfc',
    '#cbd8f9',
    '#aec4f6',
    '#90b0f4',
    '#7199f3',
    '#4f80f2',
    '#2a62f4',
    '#0433ff',    
]
tempgrad_hovmoller_plot = alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T").title("date"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q").title(['hour of day']).scale(reverse=True).axis(values=[0,6,12,18,24]),
        alt.Y2("hour_end:Q"),
        alt.Color("value:O").scale(domain=domain, range=range).title(['Static Stability', '(dθᵥ /dz, ˚C/m)'])
).properties(height = 100, width=500)
tempgrad_hovmoller_plot

alt.Chart(...)

## Using Ri_3m_c

In [57]:
src = tidy_df.query("variable == 'Ri_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()
# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
src['value'] = src['value'].round(1)

In [58]:
alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q"),
        alt.Y2("hour_end:Q"),
        alt.Color("value:Q").scale(scheme='blueorange', domain=[-1,1.0])
).properties(height = 100, width=500).configure_axis(grid=False)

alt.Chart(...)

## Using $\Omega_{3m}^c$

In [59]:
src = tidy_df.query("variable == 'omega_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()
# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
src['value'] = src['value'].round(1)

In [60]:
alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q"),
        alt.Y2("hour_end:Q"),
        alt.Color("value:Q").scale(scheme='blueorange', domain=[0, .86], clamp=True),
).properties(height = 100, width=500).configure_axis(grid=False)

alt.Chart(...)

In [61]:
src = tidy_df[tidy_df.variable.isin([
    'omega_3m_c',
    'Ri_3m_c',
    'temp_gradient_3m_c',
])][['time', 'variable', 'value']].pivot(index='time', columns='variable')
src.columns = src.columns.droplevel(0)

coupled_rule = alt.Chart(pd.DataFrame({'x':[0.43]})).mark_rule().encode(y='x')

base_chart = alt.Chart(
    src.groupby(pd.Grouper(freq='60Min')).median()
).mark_circle(
    size=5,
    opacity=0.25
)
alt.Chart(
    src.groupby(pd.Grouper(freq='60Min')).median()
).mark_circle(
    size=10,
    opacity=0.25
).properties(width = 200, height = 200)
omega_yaxis = alt.Y('omega_3m_c:Q').scale(domain=[0,5], clamp=True)
ri_xaxis = alt.X("Ri_3m_c:Q").scale(domain=[-0.5,0.5], clamp=True)

omega_vs_ri = base_chart.encode(
    ri_xaxis,
    omega_yaxis,
) + coupled_rule
tempgrad_vs_ri = base_chart.encode(
    ri_xaxis,
    alt.Y('temp_gradient_3m_c:Q'),
)
omega_vs_tempgrad = base_chart.encode(
    alt.X("temp_gradient_3m_c:Q"),
    omega_yaxis,
) + coupled_rule
omega_vs_ri | tempgrad_vs_ri | omega_vs_tempgrad

alt.HConcatChart(...)

# Make stability regime time series plot

In [62]:
def temp_gradient_to_stability_regime(x):
    if np.isnan(x):
        return None
    elif x < -0.01:
        return "unstable"
    elif x >= -0.01 and x <= 0.01:
        return "neutral"
    elif x > 0.01:
        return "stable"
    else:
        raise ValueError("what?")
src = tidy_df.query("variable == 'temp_gradient_3m_c'")
src['date'] = src.time.dt.date
src[src.time.dt.hour.isin([0,1])]
src_day = src[src.time.dt.hour.isin([12,13])].assign(time_of_day = 'day')
src_night = src[src.time.dt.hour.isin([0,1])].assign(time_of_day = 'night')
src = pd.concat([src_day, src_night])
src = src.groupby(['date', 'time_of_day']).mean(numeric_only=True).reset_index()
src['stability regime'] = src['value'].apply(temp_gradient_to_stability_regime)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_97466/2887523614.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src['date'] = src.time.dt.date


In [63]:
stability_regime_chart = alt.Chart(src).mark_bar().encode(
    alt.X("date:T"),
    alt.Color("stability regime:N").scale(domain = ['neutral', 'stable', 'unstable'], range=['#000000', '#1f77b4', '#ff7f0e']  ),
    alt.Y("time_of_day:N", sort=['night', 'day'], title=None)
).properties(width = 500)

In [101]:
src = tidy_df[tidy_df.measurement.isin([
    'virtual temperature',
    'surface virtual temperature'
])]
src = src.set_index("time").groupby([pd.Grouper(freq='60Min'), 'height']).mean(numeric_only=True).reset_index()
src['hour'] = src.time.dt.hour
src = src[src['hour']%4 == 0]

def profile_chart(src):
    return alt.Chart(src).mark_line().encode(
        alt.X("value:Q").sort('y').title("Virtual temperature (˚C)"),
        alt.Y("height:Q").title("height (m)"),
        alt.Color('hour:O').scale(scheme='rainbow')
    ).properties(width = 200, height = 125)

In [102]:
profiles_chart = (
    profile_chart(
        src[
            src.time.dt.date == dt.date(2023, 2, 11)
        ].dropna().query("height != 15")
    ).properties(title = "Sunny, midwinter day (2023-02-11)")
    |
    profile_chart(
        src[
            src.time.dt.date == dt.date(2023, 3, 28)
        ].dropna().query("height != 15")
    ).properties(title = "Sunny, spring day (2023-03-28)")
).resolve_scale(x='shared', y='shared')
profiles_chart

alt.HConcatChart(...)

In [66]:
seasonal_conditions_figure = (    
    (
        rnet_chart.properties(height = 100) & 
        daily_sub_chart & 
        tempgrad_hovmoller_plot
    ).resolve_scale(color='independent', x='shared')
    &
    profiles_chart
)
seasonal_conditions_figure.save("seasonal_conditions_figure.png", ppi=200)
seasonal_conditions_figure

alt.VConcatChart(...)

In [67]:
grad_exam_chart = (    
    (
        cum_sub_chart_no_xaxislabels.properties(height = 200) & 
        daily_sub_chart & 
        rnet_chart.properties(height = 75) & 
        tempgrad_hovmoller_plot
    ).resolve_scale(color='independent', x='shared')
).configure_legend(symbolSize=50, labelFontSize=10, rowPadding=1)

grad_exam_chart.save("gradexam_seasonal_conditions_figure.png", ppi=200)

In [69]:
wsc_chart = (    
    (
        (snowpillow_chart + cum_sub_chart_no_xaxislabels).resolve_scale(y='independent').properties(
            height = 200, width=500) & 
        daily_sub_chart.properties(height = 75) & 
        rnet_chart.properties(height = 75, width=500) & 
        tempgrad_hovmoller_plot
    ).resolve_scale(color='independent', x='shared')
    &
    profiles_chart
).configure_legend(symbolSize=50, labelFontSize=10, rowPadding=1)

wsc_chart.save("../../figures/seasonal_sublimation_and_conditions.png", ppi=200)
wsc_chart

alt.VConcatChart(...)

In [80]:
measured_results_mm.loc['20230211'].sum()

variable
w_h2o__10m_c    -0.085020
w_h2o__10m_d    -0.049890
w_h2o__10m_ue   -0.081847
w_h2o__10m_uw   -0.040274
w_h2o__15m_c    -0.028703
w_h2o__20m_c    -0.059075
w_h2o__2m_c      0.010369
w_h2o__3m_c     -0.001110
w_h2o__3m_d      0.002550
w_h2o__3m_ue    -0.000242
w_h2o__3m_uw     0.004685
w_h2o__5m_c     -0.015507
dtype: float64

In [104]:
measured_results_mm.loc['20230328'].sum()

variable
w_h2o__10m_c     0.079284
w_h2o__10m_d     0.057603
w_h2o__10m_ue    0.009182
w_h2o__10m_uw    0.100327
w_h2o__15m_c     0.048912
w_h2o__20m_c    -0.045614
w_h2o__2m_c      0.113117
w_h2o__3m_c      0.147482
w_h2o__3m_d      0.157362
w_h2o__3m_ue     0.152143
w_h2o__3m_uw     0.153702
w_h2o__5m_c      0.171921
dtype: float64

In [116]:
(
    alt.Chart(
        tidy_df.query("measurement == 'snow flux'").query("time < '20230118'").query    ("time > '20230117'")
    ).mark_circle().encode(
        alt.X('time:T'),
        alt.Y('value:Q').title('BS flux (g/m^2/s)'),
        alt.Color('height:O').scale(scheme='turbo')
    ).properties(height = 100, width=200) & alt.Chart(
        tidy_df[tidy_df.variable.isin([
            'w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c', 'w_h2o__15m_c', 'w_h2o__20m_c'
        ])].query("time < '20230118'").query("time > '20230117'")
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('value:Q').title("w'q' (g/m^2/s)").scale(domain=[-0.0001,0.1], clamp=True),
        alt.Color('height:O').scale(scheme='turbo'),
        tooltip='height'
    ).properties(height = 100, width=200) & 
    alt.Chart(
        tidy_df[tidy_df.variable.isin([
            'w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c', 'w_h2o__15m_c', 'w_h2o__20m_c'
        ])].query("time < '20230118'").query    ("time > '20230117'")
    ).transform_window(
            sort=[{'field': 'time'}],
        # Include all previous records before the current record and none after
        # (This is the default value so you could skip it and it would still work.)
        frame=[None, 0],
        # What to add up as you go
        cumulative_flux_density='sum(value)',
        groupby=['height']
    ).transform_calculate(
        cumulative_sub_mm = 'datum.cumulative_flux_density * 1800 / 1000'
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('cumulative_sub_mm:Q').title(['Cumulative', 'sublimation (mm)']),
        alt.Color('height:O').scale(scheme='turbo')
).properties(height = 100, width=200)
)  |\
(
    alt.Chart(
    tidy_df[tidy_df.variable.isin([
        'ldiag_3m_c', 'ldiag_5m_c', 'ldiag_10m_c', 'ldiag_15m_c', 'ldiag_20m_c'
    ])].query("time < '20230118'").query("time > '20230117'")
).mark_line().transform_calculate(
    percent_flagged = "100 * datum.value/6000"
).encode(
    alt.X('time:T'),
    alt.Y('percent_flagged:Q').title(['sonic diagnostic', '(% flagged)'] ),
    alt.Color('height:O').scale(scheme='turbo'),
    tooltip='height'
).properties(height = 100, width=200) &
    alt.Chart(
        tidy_df[tidy_df.variable.isin([
            'irgadiag_3m_c', 'irgadiag_5m_c', 'irgadiag_10m_c', 'irgadiag_15m_c', 'irgadiag_20m_c'
        ])].query("time < '20230118'").query("time > '20230117'")
    ).mark_line().transform_calculate(
        percent_flagged = "100 * datum.value/6000"
    ).encode(
        alt.X('time:T'),
        alt.Y('percent_flagged:Q').title(['irgason diagnostic', '(% flagged)'] ),
        alt.Color('height:O').scale(scheme='turbo'),
        tooltip='height'
    ).properties(height = 100, width=200) & alt.Chart(
        tidy_df[tidy_df.variable.isin([
            'spd_3m_c', 'spd_5m_c', 'spd_10m_c', 'spd_20m_c'
        ])].query("time < '20230118'").query("time > '20230117'")
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('value:Q').title('wind speed (m/s)'),
        alt.Color('height:O').scale(scheme='turbo'),
        tooltip='height'
    ).properties(height = 100, width=200)
)

alt.HConcatChart(...)

In [115]:
tidy_df[tidy_df.measurement == 'wind speed'].groupby('variable').max()

,time,value,height,tower,measurement
variable,,,,,
spd_10m_c,2023-05-08 23:30:00,14.158528,10.0,c,wind speed
spd_10m_d,2023-05-08 23:30:00,13.541412,10.0,d,wind speed
spd_10m_ue,2023-05-08 23:30:00,14.289142,10.0,ue,wind speed
spd_10m_uw,2023-05-08 23:30:00,13.574902,10.0,uw,wind speed
spd_15m_c,2023-05-08 23:30:00,13.951708,15.0,c,wind speed
spd_1m_c,2023-05-08 23:30:00,9.333563,1.0,c,wind speed
spd_1m_d,2023-05-08 23:30:00,49.778217,1.0,d,wind speed
spd_1m_ue,2023-05-08 23:30:00,8.141444,1.0,ue,wind speed
spd_1m_uw,2023-05-08 23:30:00,6.084682,1.0,uw,wind speed


In [114]:
tidy_df[tidy_df.measurement == 'wind speed'].set_index('time').loc['20221222'].groupby('variable').max()

,value,height,tower,measurement
variable,,,,
spd_10m_c,14.158528,10.0,c,wind speed
spd_10m_d,13.541412,10.0,d,wind speed
spd_10m_ue,14.289142,10.0,ue,wind speed
spd_10m_uw,13.574902,10.0,uw,wind speed
spd_15m_c,13.951708,15.0,c,wind speed
spd_1m_c,9.333563,1.0,c,wind speed
spd_1m_d,49.778217,1.0,d,wind speed
spd_1m_ue,8.141444,1.0,ue,wind speed
spd_1m_uw,NaN,1.0,uw,wind speed


In [105]:
src = measured_results_cumsum.groupby(['height', 'tower'])[['value']].max().reset_index()
sublimation_totals_per_height_tower_chart = alt.Chart(src).mark_point(size=100).encode(
    alt.Y("height:O").sort('-y').title("height (m)"),
    alt.X("value:Q").scale(zero=False).title(["Seasonal sublimation (mm SWE)"]),
    # alt.Color("height:O").scale(scheme='turbo').legend(columns=2),
    alt.Shape("tower:N")
).properties(width = 150, height = 150)
sublimation_totals_per_height_tower_chart.save('../../figures/sublimation_per_tower_and_height.png', ppi = 200)
sublimation_totals_per_height_tower_chart

alt.Chart(...)

In [106]:
src.round(1).sort_values('value')

,height,tower,value
11,20,c,26.3
10,15,c,29.3
8,10,ue,30.8
0,2,c,31.0
4,3,uw,33.7
2,3,d,33.9
3,3,ue,34.6
1,3,c,35.8
7,10,d,35.8
9,10,uw,36.6


In [125]:
print(
    'Dec 1-31\t',
    round(tidy_df.query("variable == 'w_h2o__10m_c'").set_index('time').loc[
        '20221201': '20221231'
    ].value.sum()*1800/density_water.magnitude, 2)
)
print(
    'Dec 12-14\t',
    round(tidy_df.query("variable == 'w_h2o__10m_c'").set_index('time').loc[
        '20221212': '20221214'
    ].value.sum()*1800/density_water.magnitude, 2)
)
print(
    'Dec 21-22\t',
    round(tidy_df.query("variable == 'w_h2o__10m_c'").set_index('time').loc[
        '20221221': '20221222'
    ].value.sum()*1800/density_water.magnitude, 2)
)

Dec 1-31	 6.63
Dec 12-14	 1.31
Dec 21-22	 2.09


In [17]:
# Identify lists of timestamps for different categories
bs_times = set(
    tidy_df.query("variable == 'SF_avg_1m_ue'").query("value > 0").time
).union(
    set(tidy_df.query("variable == 'SF_avg_2m_ue'").query("value > 0").time)
)
nobs_times = set(tidy_df.time).difference(bs_times)

decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

upvalley_wind_times = tidy_df[tidy_df.variable == 'dir_3m_c'].query("value < 152").query("value > 112").time.values
downvalley_wind_times = tidy_df[tidy_df.variable == 'dir_3m_c'].query("value < 332").query("value > 292").time.values
len(upvalley_wind_times),len(downvalley_wind_times)

(1114, 3066)

In [49]:
measured_results_sum = measured_results_mm.sum().rename('sum_sublimation').reset_index()
measured_results_sum['height'] = measured_results_sum['variable'].apply(lambda s: int(s.split('__')[1].split('m')[0]))
measured_results_sum['tower'] = measured_results_sum['variable'].apply(lambda s: s.split('__')[1].split('_')[-1])
measured_results_sum['tower-height'] = measured_results_sum['tower'] + '-' + measured_results_sum['height'].astype('str')
measured_results_sum

measured_results_sum_bs = measured_results_mm[measured_results_mm.index.isin(bs_times)].sum().rename('sum_sublimation').reset_index()
measured_results_sum_bs['height'] = measured_results_sum_bs['variable'].apply(lambda s: int(s.split('__')[1].split('m')[0]))
measured_results_sum_bs['tower'] = measured_results_sum_bs['variable'].apply(lambda s: s.split('__')[1].split('_')[-1])
measured_results_sum_bs['tower-height'] = measured_results_sum_bs['tower'] + '-' + measured_results_sum_bs['height'].astype('str')
measured_results_sum_bs

measured_results_sum_nobs = measured_results_mm[measured_results_mm.index.isin(nobs_times)].sum().rename('sum_sublimation').reset_index()
measured_results_sum_nobs['height'] = measured_results_sum_nobs['variable'].apply(lambda s: int(s.split('__')[1].split('m')[0]))
measured_results_sum_nobs['tower'] = measured_results_sum_nobs['variable'].apply(lambda s: s.split('__')[1].split('_')[-1])
measured_results_sum_nobs['tower-height'] = measured_results_sum_nobs['tower'] + '-' + measured_results_sum_nobs['height'].astype('str')
measured_results_sum_nobs

,variable,sum_sublimation,height,tower,tower-height
0,w_h2o__10m_c,14.577066,10,c,c-10
1,w_h2o__10m_d,16.157052,10,d,d-10
2,w_h2o__10m_ue,13.741472,10,ue,ue-10
3,w_h2o__10m_uw,14.970027,10,uw,uw-10
4,w_h2o__15m_c,10.331743,15,c,c-15
5,w_h2o__20m_c,9.677669,20,c,c-20
6,w_h2o__2m_c,15.333160,2,c,c-2
7,w_h2o__3m_c,17.297341,3,c,c-3
8,w_h2o__3m_d,16.331905,3,d,d-3
9,w_h2o__3m_ue,16.512565,3,ue,ue-3


In [51]:
base = alt.Chart(
    
).mark_point(size=100).encode(
    alt.Y("height:O").sort('-y').title("height (m)"),
    alt.X("sum_sublimation:Q").scale(zero=False).title(["Seasonal sublimation (mm SWE)"]),
    # alt.Color("height:O").scale(scheme='turbo').legend(columns=2),
    alt.Shape("tower:N"),
    tooltip='sum_sublimation'
).properties(width = 200, height = 200)

alt.layer(base, data = measured_results_sum).properties(title = 'All data') &\
alt.layer(base, data = measured_results_sum_nobs).properties(title = 'No Blowing Snow') &\
alt.layer(base, data = measured_results_sum_bs).properties(title = 'Blowing Snow')

alt.VConcatChart(...)

In [36]:
src = tidy_df[tidy_df.variable.isin([
    'w_h2o__10m_c',
    'w_h2o__10m_d',
    'w_h2o__10m_ue',
    'w_h2o__10m_uw',
    'w_h2o__15m_c',
    'w_h2o__1m_c',
    'w_h2o__1m_d',
    'w_h2o__1m_ue',
    'w_h2o__1m_uw',
    'w_h2o__20m_c',
    'w_h2o__2_5m_uw',
    'w_h2o__2m_c',
    'w_h2o__2m_d',
    'w_h2o__2m_ue',
    'w_h2o__2m_uw',
    'w_h2o__3m_c',
    'w_h2o__3m_d',
    'w_h2o__3m_ue',
    'w_h2o__3m_uw',
    'w_h2o__5m_c'
])]


KeyboardInterrupt: 

In [ ]:
alt.Chart(
    src[
        (src.time >= '2022-12-21 1800') &
        (src.time <= '2022-12-21 1800')
    ].dropna()
).mark_line(point=True).encode(
    alt.X("value:Q").sort('-y').scale(domain=[0, 0.06], clamp=True).title(
        "w'q' (g/m^2/s)"
    ),
    alt.Y("height:Q"),
    alt.Color("tower:N"),
    alt.Facet("time:T", columns=6).header(format='%H:%M')
).properties(width = 150, height=150)

In [ ]:
alt.Chart(
    src[
        (src.time >= '2022-12-21 1800') &
        (src.time <= '2022-12-21 2200') 
    ].dropna()
).mark_line(point=True).encode(
    alt.X("value:Q").sort('-y').scale(domain=[0, 0.06], clamp=True).title(
        "w'q' (g/m^2/s)"
    ),
    alt.Y("height:Q"),
    alt.Color("tower:N"),
    alt.Facet("time:T", columns=6).header(format='%H:%M')
).properties(width = 150, height=150)

In [ ]:
alt.Chart(tidy_df.query("variable == 'SnowDepth_c'")).mark_line().encode(
    alt.X("time:T").title(None),
    alt.Y("value:Q").title("Snow depth (m)")
).properties(width = 150, height = 150)

In [ ]:
src_swe = tidy_df[
    tidy_df.variable.isin(['SWE_p3_c', 'SWE_p2_c', 'SWE_p1_c', 'SWE_p4_c'])
][
    (tidy_df.time > "20221220 2300") & (tidy_df.time < "20221223 0100")
]
src_swe.value = src_swe.value/1000


src_bs = tidy_df[
    tidy_df.variable.isin(['SF_avg_1m_ue'])
][
    (tidy_df.time > "20221220 2300") & (tidy_df.time < "20221223 0100")
]


src_lhflux = tidy_df[
    tidy_df.variable.isin(['w_h2o__10m_c'])
][
    (tidy_df.time > "20221220 2300") & (tidy_df.time < "20221223 0100")
]
src_lhflux = src_lhflux.set_index('time')
src_lhflux.loc[
    "2022-12-22 03:30:00": "2022-12-22 07:30:00",
    ['value']
] = np.nan
src_lhflux.loc[
    "2022-12-21 23:30:00",
    ['value']
] = np.nan
src_lhflux.value = src_lhflux.value.interpolate(method='cubicspline')

In [55]:
chart_swe = alt.Chart(
    src_swe[(src_swe.time.dt.hour%4==0) & (src_swe.time.dt.minute == 0)]
    #[src_swe.variable != 'SWE_p1_c']
).mark_line(point=True, strokeWidth=1).encode(
    alt.X("time:T").title(None).axis(format='%m/%d', tickCount='day'),
    alt.Y("value:Q").title(['SWE (m)', 'from four', 'snow', 'pillows']).scale(domain = [0.00, 0.15]),
    alt.Color("variable:N").sort(['SWE_p3_c', 'SWE_p2_c', 'SWE_p4_c', 'SWE_p1_c', ]),
    alt.Shape("variable:N").sort(['SWE_p3_c', 'SWE_p2_c', 'SWE_p4_c', 'SWE_p1_c', ])
).properties(
    width = 150, 
    height = 75
)
chart_lhflux = alt.Chart(
    src_lhflux.reset_index()
).mark_line().encode(
    alt.X("time:T").title(None).axis(format='%m/%d', tickCount='day'),
    alt.Y("value:Q").title(['Sublimation', '(g m⁻² s⁻¹)', 'from Eddy', 'Covariance'])
).properties(
    width = 150, 
    height = 75
)
chart_bs = alt.Chart(
    src_bs.reset_index()
).mark_line().encode(
    alt.X("time:T").title(None).axis(format='%m/%d', tickCount='day'),
    alt.Y("value:Q").title(['Horizontal', 'blowing', 'snow flux', '(g m⁻² s⁻¹)'])
).properties(
    width = 150, 
    height = 75
)
(chart_swe & chart_bs & chart_lhflux).configure_axisY(
    titleAngle=0
)


alt.VConcatChart(...)

# Calculate how frequently blowing snow occurred

In [ ]:
[v for v in tidy_df.variable.unique() if v.startswith('SF')] 

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
blowing_snow_src = tidy_df[
    tidy_df.variable.isin(['SF_avg_2m_ue', 'SF_avg_1m_ue'])
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

blowing_snow_src.columns = blowing_snow_src.columns.droplevel(0)
blowing_snow_src['SF_avg_ue'] = blowing_snow_src['SF_avg_1m_ue'] + blowing_snow_src['SF_avg_2m_ue']

# Calculate how much sublimation occurred during blowing snow

In [ ]:
print("Whole Season Statistics")
fraction_time_with_blowing_snow = len( 
    blowing_snow_src.query("SF_avg_ue > 0")
) / len(
    blowing_snow_src
)

print(f"Blowing snow occurred {round(fraction_time_with_blowing_snow*100, 1)}% of the time")

times_with_blowing_snow = blowing_snow_src.query("SF_avg_ue > 0").index.values

min_lh_fluxes_during_bs = tidy_df.query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_during_bs = min_lh_fluxes_during_bs[min_lh_fluxes_during_bs.time.isin(times_with_blowing_snow)]
min_lh_fluxes_no_bs = tidy_df.query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_no_bs = min_lh_fluxes_no_bs[ ~ min_lh_fluxes_no_bs.time.isin(times_with_blowing_snow)]

mm_sublimation_during_bs = (min_lh_fluxes_during_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_no_bs = (min_lh_fluxes_no_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = (mm_sublimation_during_bs + mm_sublimation_no_bs)

print("total sublimaton at 3m_c")
print(mm_sublimation_total)
print("during bs sublimaton at 3m_c, not during bs")
print(mm_sublimation_during_bs, mm_sublimation_no_bs)
print("respective percentages")
print(
    round(100*mm_sublimation_during_bs/mm_sublimation_total, 1), 
    round(100*mm_sublimation_no_bs/mm_sublimation_total, 1)
)


In [ ]:
print("Pre-Spring Equinox Statistics")

fraction_time_with_blowing_snow = len( 
    blowing_snow_src[: "20230320"].query("SF_avg_ue > 0")
) / len(
    blowing_snow_src[: "20230320"]
)

print(f"Blowing snow occurred {round(fraction_time_with_blowing_snow*100, 1)}% of the time")

times_with_blowing_snow = blowing_snow_src[: "20230320"].query("SF_avg_ue > 0").index.values

min_lh_fluxes_during_bs = tidy_df[tidy_df.time < "20230320"].query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_during_bs = min_lh_fluxes_during_bs[min_lh_fluxes_during_bs.time.isin(times_with_blowing_snow)]
min_lh_fluxes_no_bs = tidy_df[tidy_df.time < "20230320"].query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_no_bs = min_lh_fluxes_no_bs[ ~ min_lh_fluxes_no_bs.time.isin(times_with_blowing_snow)]

mm_sublimation_during_bs = (min_lh_fluxes_during_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_no_bs = (min_lh_fluxes_no_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = (mm_sublimation_during_bs + mm_sublimation_no_bs)

print("total sublimaton at 3m_c")
print(mm_sublimation_total)
print("during bs sublimaton at 3m_c, not during bs")
print(mm_sublimation_during_bs, mm_sublimation_no_bs)
print("respective percentages")
print(
    round(100*mm_sublimation_during_bs/mm_sublimation_total, 1), 
    round(100*mm_sublimation_no_bs/mm_sublimation_total, 1)
)


In [ ]:
print("Post-Spring Equinox Statistics")

fraction_time_with_blowing_snow = len( 
    blowing_snow_src["20230320": ].query("SF_avg_ue > 0")
) / len(
    blowing_snow_src["20230320": ]
)

print(f"Blowing snow occurred {round(fraction_time_with_blowing_snow*100, 1)}% of the time")

times_with_blowing_snow = blowing_snow_src["20230320": ].query("SF_avg_ue > 0").index.values

min_lh_fluxes_during_bs = tidy_df[tidy_df.time >= "20230320"].query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_during_bs = min_lh_fluxes_during_bs[min_lh_fluxes_during_bs.time.isin(times_with_blowing_snow)]
min_lh_fluxes_no_bs = tidy_df[tidy_df.time >= "20230320"].query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_no_bs = min_lh_fluxes_no_bs[ ~ min_lh_fluxes_no_bs.time.isin(times_with_blowing_snow)]

mm_sublimation_during_bs = (min_lh_fluxes_during_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_no_bs = (min_lh_fluxes_no_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = (mm_sublimation_during_bs + mm_sublimation_no_bs)

print("total sublimaton at 3m_c")
print(mm_sublimation_total)
print("during bs sublimaton at 3m_c, not during bs")
print(mm_sublimation_during_bs, mm_sublimation_no_bs)
print("respective percentages")
print(
    round(100*mm_sublimation_during_bs/mm_sublimation_total, 1), 
    round(100*mm_sublimation_no_bs/mm_sublimation_total, 1)
)


# Calculate how frequently different stability regimes occurred

## Using static stability, $\frac{d\theta_v}{dz}$ at 3m 

In [ ]:
def temp_gradient_to_stability_regime(x, threshold = 0.01):
    if np.isnan(x):
        return None
    elif x < -threshold:
        return "unstable"
    elif x >= -threshold and x <= threshold:
        return "neutral"
    elif x > threshold:
        return "stable"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
stab_regimes_src = tidy_df[
    tidy_df.variable == 'temp_gradient_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

stab_regimes_src.columns = stab_regimes_src.columns.droplevel(0)
stab_regimes_src['stab_regime_3m_c'] = stab_regimes_src['temp_gradient_3m_c'].apply(lambda v: temp_gradient_to_stability_regime(v, 0.01))
print(stab_regimes_src['stab_regime_3m_c'].value_counts()/len(stab_regimes_src['stab_regime_3m_c'].dropna()))

## Using dynamic stability, $Ri$

In [ ]:
def ri_to_stability_regime(x, unstable_threshold = -0.01, stable_threshold=0.25):
    if np.isnan(x):
        return None
    elif x < unstable_threshold:
        return "unstable"
    elif x >= unstable_threshold and x <= stable_threshold:
        return "neutral"
    elif x > stable_threshold:
        return "stable"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
dynamicstab_regimes_src = tidy_df[
    tidy_df.variable == 'Ri_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

dynamicstab_regimes_src.columns = dynamicstab_regimes_src.columns.droplevel(0)
dynamicstab_regimes_src['stab_regime_3m_c'] = dynamicstab_regimes_src['Ri_3m_c'].apply(lambda v: ri_to_stability_regime(v, 0.01))
print(dynamicstab_regimes_src['stab_regime_3m_c'].value_counts()/len(dynamicstab_regimes_src['stab_regime_3m_c'].dropna()))

## Using coupling parameter, $\Omega$

In [ ]:
def omega_to_coupling_regime(x, weakly_coupled_threshold = 0.43, strongly_coupled_threshold=0.61):
    if np.isnan(x):
        return None
    elif x < weakly_coupled_threshold:
        return "uncoupled"
    elif x >= weakly_coupled_threshold and x <= strongly_coupled_threshold:
        return "weakly coupled"
    elif x > strongly_coupled_threshold:
        return "strongly coupled"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
coupling_regimes_src = tidy_df[
    tidy_df.variable == 'omega_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

coupling_regimes_src.columns = coupling_regimes_src.columns.droplevel(0)
coupling_regimes_src['omega_regime_3m_c'] = coupling_regimes_src['omega_3m_c'].apply(lambda v: omega_to_coupling_regime(v))
print(coupling_regimes_src['omega_regime_3m_c'].value_counts()/len(coupling_regimes_src['omega_regime_3m_c'].dropna()))

# Calculate how much sublimation occured during different stability regimes

## static stability

In [ ]:
times_w_stable = stab_regimes_src.query("stab_regime_3m_c == 'stable'").index.values
times_w_unstable = stab_regimes_src.query("stab_regime_3m_c == 'unstable'").index.values
times_w_neutral = stab_regimes_src.query("stab_regime_3m_c == 'neutral'").index.values

lhfluxes = tidy_df.query("variable == 'w_h2o__3m_c'")

lhdluxes_stable = lhfluxes[lhfluxes.time.isin(times_w_stable)]
lhdluxes_unstable = lhfluxes[lhfluxes.time.isin(times_w_unstable)]
lhdluxes_neutral = lhfluxes[lhfluxes.time.isin(times_w_neutral)]

In [ ]:
mm_sublimation_stable = (lhdluxes_stable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_unstable = (lhdluxes_unstable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_neutral = (lhdluxes_neutral['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = mm_sublimation_stable + mm_sublimation_unstable + mm_sublimation_neutral
print('stable', 'neutral', 'unstable')
print(mm_sublimation_stable, mm_sublimation_neutral, mm_sublimation_unstable)
print(
    mm_sublimation_stable/mm_sublimation_total, 
    mm_sublimation_neutral/mm_sublimation_total,
    mm_sublimation_unstable/mm_sublimation_total, 
)
print(mm_sublimation_total)

# dynamic stability

In [ ]:
times_w_stable = dynamicstab_regimes_src.query("stab_regime_3m_c == 'stable'").index.values
times_w_unstable = dynamicstab_regimes_src.query("stab_regime_3m_c == 'unstable'").index.values
times_w_neutral = dynamicstab_regimes_src.query("stab_regime_3m_c == 'neutral'").index.values

lhfluxes = tidy_df.query("variable == 'w_h2o__3m_c'")

lhdluxes_stable = lhfluxes[lhfluxes.time.isin(times_w_stable)]
lhdluxes_unstable = lhfluxes[lhfluxes.time.isin(times_w_unstable)]
lhdluxes_neutral = lhfluxes[lhfluxes.time.isin(times_w_neutral)]

In [ ]:
mm_sublimation_stable = (lhdluxes_stable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_unstable = (lhdluxes_unstable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_neutral = (lhdluxes_neutral['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = mm_sublimation_stable + mm_sublimation_unstable + mm_sublimation_neutral
print('stable', 'neutral', 'unstable')
print(mm_sublimation_stable, mm_sublimation_neutral, mm_sublimation_unstable)
print(
    mm_sublimation_stable/mm_sublimation_total, 
    mm_sublimation_neutral/mm_sublimation_total,
    mm_sublimation_unstable/mm_sublimation_total, 
)
print(mm_sublimation_total)

# Coupling parameter

In [ ]:
times_decoupled = coupling_regimes_src.query("omega_regime_3m_c == 'uncoupled'").index.values
times_weaklycoupled = coupling_regimes_src.query("omega_regime_3m_c == 'weakly coupled'").index.values
times_coupled = coupling_regimes_src.query("omega_regime_3m_c == 'strongly coupled'").index.values

lhdluxes_decoupled = lhfluxes[lhfluxes.time.isin(times_decoupled)]
lhdluxes_weaklycoupled = lhfluxes[lhfluxes.time.isin(times_weaklycoupled)]
lhdluxes_coupled = lhfluxes[lhfluxes.time.isin(times_coupled)]

In [ ]:
mm_sublimation_decoupled = (lhdluxes_decoupled['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_weaklycoupled = (lhdluxes_weaklycoupled['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_coupled = (lhdluxes_coupled['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()

mm_sublimation_total = mm_sublimation_decoupled + mm_sublimation_weaklycoupled + mm_sublimation_coupled
print('uncoupled', 'weakly coupled', 'strongly coupled')
print(
    mm_sublimation_decoupled, mm_sublimation_weaklycoupled, mm_sublimation_coupled
)
print(
    mm_sublimation_decoupled/mm_sublimation_total, 
    mm_sublimation_weaklycoupled/mm_sublimation_total,
    mm_sublimation_coupled/mm_sublimation_total, 
)
print(mm_sublimation_total)

In [ ]:
static
stable neutral unstable
30.0130214793065 3.8164881851337578 2.34608864842682
0.8296482402236137 0.10549896513463593 0.0648527946417504
36.17559831286707

dynamic
stable neutral unstable
4.463500869073066 20.722328462239695 10.550510629803817
0.12490089566893645 0.5798671180313069 0.2952319862997567
35.73633996111658

coupling
uncoupled weakly coupled strongly coupled
-0.8071439238296523 0.38557088660657446 29.636504509509173
-0.027627787680944114 0.013197733733256748 1.0144300539476874
29.214931472286096